<a href="https://colab.research.google.com/github/biranchi2018/My-NLP-Examples/blob/master/6.Transformers_Fill_Mask_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 665kB 2.8MB/s 
     |████████████████████████████████| 3.8MB 13.6MB/s 
     |████████████████████████████████| 1.1MB 29.2MB/s 
     |████████████████████████████████| 890kB 47.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=403f2f5d2095f67e9c2594f7cf88a87342fe56105b2ef784d338b41841ab89af
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
from transformers import pipeline

In [4]:
nlp_fill = pipeline('fill-mask')
result = nlp_fill('Do ' + nlp_fill.tokenizer.mask_token + ' every day and you will ')
result

[{'score': 0.21916480362415314,
  'sequence': '<s> Do it every day and you will</s>',
  'token': 24},
 {'score': 0.10750742256641388,
  'sequence': '<s> Do so every day and you will</s>',
  'token': 98},
 {'score': 0.06556503474712372,
  'sequence': '<s> Do this every day and you will</s>',
  'token': 42},
 {'score': 0.060048460960388184,
  'sequence': '<s> Do yoga every day and you will</s>',
  'token': 11025},
 {'score': 0.04253779724240303,
  'sequence': '<s> Do something every day and you will</s>',
  'token': 402}]

In [12]:
result = nlp_fill('Do it every day and you will ' + nlp_fill.tokenizer.mask_token + '!')
result

[{'score': 0.19267874956130981,
  'sequence': '<s> Do it every day and you will succeed!</s>',
  'token': 6726},
 {'score': 0.13028372824192047,
  'sequence': '<s> Do it every day and you will win!</s>',
  'token': 339},
 {'score': 0.08598393201828003,
  'sequence': '<s> Do it every day and you will learn!</s>',
  'token': 1532},
 {'score': 0.04379066452383995,
  'sequence': '<s> Do it every day and you will prosper!</s>',
  'token': 30435},
 {'score': 0.04223969206213951,
  'sequence': '<s> Do it every day and you will enjoy!</s>',
  'token': 2254}]

In [6]:
result = nlp_fill('Do yoga every day and you will ' + nlp_fill.tokenizer.mask_token + '!')
result

[{'score': 0.1741621345281601,
  'sequence': '<s> Do yoga every day and you will succeed!</s>',
  'token': 6726},
 {'score': 0.09435515105724335,
  'sequence': '<s> Do yoga every day and you will improve!</s>',
  'token': 1477},
 {'score': 0.09162674844264984,
  'sequence': '<s> Do yoga every day and you will learn!</s>',
  'token': 1532},
 {'score': 0.05402381345629692,
  'sequence': '<s> Do yoga every day and you will excel!</s>',
  'token': 21317},
 {'score': 0.05259063094854355,
  'sequence': '<s> Do yoga every day and you will benefit!</s>',
  'token': 1796}]

In [8]:
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 2.8MB/s 


In [0]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertForMaskedLM

In [11]:
text = (
    "Do _ every day and you will _ !"
)

# Load pre-trained model with masked language model head
bert_version = 'bert-large-uncased'
model = BertForMaskedLM.from_pretrained(bert_version)

# Preprocess text
tokenizer = BertTokenizer.from_pretrained(bert_version)
tokenized_text = tokenizer.tokenize(text)
mask_positions = []
for i in range(len(tokenized_text)):
    if tokenized_text[i] == '_':
        tokenized_text[i] = '[MASK]'
        mask_positions.append(i)

# Predict missing words from left to right
model.eval()
for mask_pos in mask_positions:
    # Convert tokens to vocab indices
    token_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([token_ids])
    # Call BERT to predict token at this position
    predictions = model(tokens_tensor)[0, mask_pos]
    predicted_index = torch.argmax(predictions).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
    # Update text
    tokenized_text[mask_pos] = predicted_token

for mask_pos in mask_positions:
    tokenized_text[mask_pos] = "_" + tokenized_text[mask_pos] + "_"
print(' '.join(tokenized_text).replace(' ##', ''))


do _for_ every day and you will _be_ !
